In [2]:
import pandas as pd
from langdetect import detect

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [1]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=39585bbe82daa6b571ff6050f1ffd2f77b698a46e10badb2b6721a645ca67ce2
  Stored in directory: /Users/danielmak/Library/Caches/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [24]:
dataset_df = pd.read_csv('historical_lead_records.csv', engine = "python")

In [25]:
dataset_df['Title'] = dataset_df['Title'].str.strip().str.upper()
dataset_df.head()

,Title,Job Role,Job Function,Job Level
0,CHIEF INFORMATION SECURITY OFFICER,Information Security,IT,C-Level
1,GROUP IT DIRECTOR,IT General,IT,Director
2,"REGIONAL HEALTH ADMINISTRATOR, REGION 7",None Technical,Medical,Contributor
3,CYBER ANALYST,Information Security,IT,Contributor
4,DIRECTOR TRANSFORMACIÓN DIGITAL,Networking,IT,Director


In [26]:
dataset_df.shape
dataset_df = dataset_df.astype(str)
dataset_df.dtypes
#dataset_df['Title'].value_counts().to_csv('title_summary_stats.csv')


(1647488, 4)

Title           object
Job Role        object
Job Function    object
Job Level       object
dtype: object

In [3]:
summary_stats_df = pd.read_csv('title_summary_stats.csv').rename(columns = {'Unnamed: 0': 'Title', 'Title': 'Counts'}).astype(str)

In [4]:
patterns_title = ['.*\?.*'
                  , 'NaN'
                  , 'nan'
                  ]

In [5]:
summary_stats_df['Title'] = summary_stats_df['Title'].str.strip()

In [6]:
# Create a combined boolean mask for rows to exclude, using regex for matching
mask = pd.concat([summary_stats_df['Title'].str.contains(pat, case=False, na=False, regex=True) for pat in patterns_title], axis=1).any(axis=1)

# Invert the mask to keep rows that do not contain any of the patterns
summary_stats_df = summary_stats_df[~mask]

In [7]:
summary_stats_df = summary_stats_df[summary_stats_df['Title'].map(lambda x: x.isascii())]

In [11]:
def detect_english(title):
    try:
        # Check if title is not empty and not just whitespace
        if pd.notnull(title) and title.strip():
            return detect(title) == 'en'
        else:
            return False
    except:
        # Handle case where language could not be detected
        return False

In [12]:


# Assuming 'data' is your DataFrame and 'Title' is the column with job titles
summary_stats_df['is_english'] = summary_stats_df['Title'].apply(detect_english)
#english_titles = summary_stats_df[summary_stats_df['is_english']]

In [7]:
summary_stats_df.head(1000)

,Title,Counts
1,CISO,23751
2,CHIEF INFORMATION OFFICER,18437
3,CIO,14891
4,DIRECTOR,14525
5,NETWORK ENGINEER,12852
6,IT MANAGER,10829
7,CHIEF INFORMATION SECURITY OFFICER,10498
8,CEO,10269
9,SECURITY ENGINEER,9222
10,"DIRECTOR, INFORMATION TECHNOLOGY",9027


,Title,Counts


In [37]:
pivot_table = pd.pivot_table(dataset_df, index=['Job Role', 'Job Function', 'Job Level'], aggfunc={'Title':'count'})
pivot_table.to_csv('pivot_table_role_function_level.csv')